## Import libraries

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
from PIL import Image, ImageFont, ImageDraw

import sys
#print(sys.path)
sys.path.append('/home/arg/SPIRAL-tensorflow/libs/mypaint')
from lib import surface, tiledsurface, brush

import utils as ut
#from . import utils
from envs import utils

#from .mypaint_utils import *
#from envs import mypaint_utils
from envs.mypaint_utils import *

#from .base import Environment
from envs.base import Environment

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


## Create environment
### Actions
> ### pressure: 2, jump: 2, size: 2
>> ### Add color
### State
> ### Coordinates?
### Reward
> Not giving anything yet

- numpy.arange([start, ]stop, [step, ]dtype=None)
- utils.uniform_locations(self.screen_size, self.location_size, 0)

In [16]:
'''
test_sizes = {
    'int': 4,
    'list1': [4],
    'list2': [4], # a max range of a list
    'any': None
}
#print(test_sizes['list'])
test1 = np.arange(0.2,1.5,0.3)
test1 = test1[:test_sizes['list1'][0]]
test3 = test1[:test_sizes['int']]

test2 = np.arange(0.8,0,-0.3)
test2 = test2[:test_sizes['list2'][0]]

print(test1)
print(test3)
print(test2)
'''

[ 0.2  0.5  0.8  1.1]
[ 0.2  0.5  0.8  1.1]
[ 0.8  0.5  0.2]


In [2]:
class MNIST(Environment):
    head = 0.25
    tail = 0.75
    
    # what's this? how many class in it
    action_sizes = {
            'pressure': [2],
            'jump': [2],
            #'color': [4],
            'size': [2],
            'control': None,
            'end': None,
    }

    size = 0.2
    pressure = 0.3

    def __init__(self, args):
        super(MNIST, self).__init__(args)
        self.mnist_nums = args.mnist_nums
        self.colorize = not args.train

        self.prepare_mnist()

        # jump
        self.jumps = [0, 1]

        # size
        self.sizes = np.arange(0.2, 2.0, 0.5)
        self.sizes = self.sizes * 1
        if 'size' in self.action_sizes:
            self.sizes = \
                    self.sizes[:self.action_sizes['size'][0]]

        # pressure
        self.pressures = np.arange(0.8, 0, -0.3)
        if 'pressure' in self.action_sizes:
            self.pressures = \
                    self.pressures[:self.action_sizes['pressure'][0]]
        # color
        self.colors = [
                (0., 0., 0.), # black
                (102., 217., 232.), # cyan 3
                (173., 181., 189.), # gray 5
                (255., 224., 102.), # yellow 3
                (229., 153., 247.), # grape 3
                (99., 230., 190.), # teal 3
                (255., 192., 120.), # orange 3
                (255., 168., 168.), # red 3
        ]
        self.colors = np.array(self.colors) / 255.

        if 'color' in self.action_sizes:
            self.colors = self.colors[:self.action_sizes['color'][0]]

        self.controls = utils.uniform_locations(
                self.screen_size, self.location_size, 0)

        self.ends = utils.uniform_locations(
                self.screen_size, self.location_size, 0)

    def reset(self):
        self.entry_pressure = np.min(self.pressures)

        if self.conditional:
            self.random_target = self.get_random_target(num=1, squeeze=True)
        else:
            self.random_target = None

        self.s = tiledsurface.Surface()
        self.s.flood_fill(0, 0, (255, 255, 255), (0, 0, 64, 64), 0, self.s)
        self.s.begin_atomic()

        with open(self.args.brush_path) as fp:
            self.bi = brush.BrushInfo(fp.read())
        self.b = brush.Brush(self.bi)

        self._step = 0
        self.s_x, self.s_y = None, None

        if self.args.conditional:
            self.z = None
        else:
            self.z = np.random.uniform(-0.5, 0.5, size=self.args.z_dim)

        return self.state, self.random_target, self.z

    def draw(self, ac, s=None, dtime=1):
        if s is None:
            s = self.s

        jump = 0
        x, y = self.ends[0]
        c_x, c_y = self.controls[0]
        color = self.colors[0]
        pressure, size = self.pressure, self.size

        # what time will it ends?
        for name in self.action_sizes:
            named_ac = ac[self.ac_idx[name]]
            value = getattr(self, name + "s")[named_ac]
            
            # action names, one action at a time
            if name == 'end':
                x, y = value
            # change drawing status, until we want to control
            if name == 'control':
                c_x, c_y = value
            elif name == 'pressure':
                pressure = value
            elif name == 'size':
                size = value
            # what jump means
            elif name == 'jump':
                jump = value

        if self.colorize:
            self.b.brushinfo.set_color_rgb(self.colors[self._step])
        if 'size' in self.action_sizes:
            self.b.brushinfo.set_base_value('radius_logarithmic', size)

        if (self.s_x is None and self.s_y is None) or \
                ('jump' in self.action_sizes and jump):
            # when self._step == 0
            self.s_x, self.s_y = 0, 0
            # when jump
            pressure = 0
            self._stroke_to(self.s_x, self.s_y, pressure)
        else:
            self._stroke_to(self.s_x, self.s_y, pressure)

        self._draw(x, y, c_x, c_y, pressure, size, color, dtime)

    def _draw(self, x, y, c_x, c_y,
              pressure, size, color, dtime):
        end_pressure = pressure

        # if straight line or jump
        if 'control' not in self.action_sizes or pressure == 0:
            self.b.stroke_to(
                    self.s.backend, x, y, pressure, 0, 0, dtime)
        else:
            end_pressure = self.curve(
                    c_x, c_y, self.s_x, self.s_y, x, y, pressure)

        self.entry_pressure = end_pressure

        self.s_x, self.s_y = x, y

        self.s.end_atomic()
        self.s.begin_atomic()

    # sx, sy = starting point
    # ex, ey = end point
    # kx, ky = curve point from last line
    # lx, ly = last point from InteractionMode update
    def curve(self, cx, cy, sx, sy, ex, ey, pressure):
        #entry_p, midpoint_p, junk, prange2, head, tail
        entry_p, midpoint_p, prange1, prange2, h, t = \
                self._line_settings(pressure)

        points_in_curve = 100
        mx, my = midpoint(sx, sy, ex, ey)
        length, nx, ny = length_and_normal(mx, my, cx, cy)
        cx, cy = multiply_add(mx, my, nx, ny, length*2)
        x1, y1 = difference(sx, sy, cx, cy)
        x2, y2 = difference(cx, cy, ex, ey)
        head = points_in_curve * h
        head_range = int(head)+1
        tail = points_in_curve * t
        tail_range = int(tail)+1
        tail_length = points_in_curve - tail

        # Beginning
        px, py = point_on_curve_1(1, cx, cy, sx, sy, x1, y1, x2, y2)
        length, nx, ny = length_and_normal(sx, sy, px, py)
        bx, by = multiply_add(sx, sy, nx, ny, 0.25)
        self._stroke_to(bx, by, entry_p)
        pressure = abs(1/head * prange1 + entry_p)
        self._stroke_to(px, py, pressure)

        for i in xrange(2, head_range):
            px, py = point_on_curve_1(i, cx, cy, sx, sy, x1, y1, x2, y2)
            pressure = abs(i/head * prange1 + entry_p)
            self._stroke_to(px, py, pressure)

        # Middle
        for i in xrange(head_range, tail_range):
            px, py = point_on_curve_1(i, cx, cy, sx, sy, x1, y1, x2, y2)
            self._stroke_to(px, py, midpoint_p)

        # End
        for i in xrange(tail_range, points_in_curve+1):
            px, py = point_on_curve_1(i, cx, cy, sx, sy, x1, y1, x2, y2)
            pressure = abs((i-tail)/tail_length * prange2 + midpoint_p)
            self._stroke_to(px, py, pressure)

        return pressure

    def _stroke_to(self, x, y, pressure, duration=0.1):
        self.b.stroke_to(
                self.s.backend,
                x, y,
                pressure,
                0.0, 0.0,
                duration)

    def get_random_target(self, num=1, squeeze=False):
        random_idxes = np.random.choice(self.real_data.shape[0], num, replace=False)
        random_image = self.real_data[random_idxes]
        if squeeze:
            random_image = np.squeeze(random_image, 0)
        return random_image

    def step(self, acs):
        self.draw(acs, self.s)
        self._step += 1
        terminal = (self._step == self.episode_length)
        if terminal:
            if self.conditional:
                reward = 1
                reward += - utils.l2(self.state, self.random_target) \
                        / np.prod(self.observation_shape)
            else:
                reward = 0
        else:
            reward = 0
        # state, reward, terminal, info
        return self.state, reward, terminal, {}

    def save_image(self, path="test.png"):
        Image.fromarray(self.image.astype(np.uint8).squeeze()).save(path)
        #self.s.save_as_png(path, alpha=False)

    @property
    def image(self):
        rect = [0, 0, self.height, self.width]
        scanline_strips = \
                surface.scanline_strips_iter(self.s, rect)
        return next(scanline_strips)

    @property
    def state(self):
        return utils.rgb2gray(self.image)

    def get_action_desc(self, ac):
        desc = []
        for name in self.action_sizes:
            named_ac = ac[self.ac_idx[name]]
            actual_ac = getattr(self, name+"s")[named_ac]
            desc.append("{}: {} ({})".format(name, actual_ac, named_ac))
        return "\n".join(desc)

    def _line_settings(self, pressure):
        p1 = self.entry_pressure
        p2 = (self.entry_pressure + pressure) / 2
        p3 = pressure
        if self.head == 0.0001:
            p1 = p2
        prange1 = p2 - p1
        prange2 = p3 - p2
        return p1, p2, prange1, prange2, self.head, self.tail

    def prepare_mnist(self):
        ut.io.makedirs(self.args.data_dir)
        
        # ground truth MNIST data
        ### where is my data_dir???
        mnist_dir = self.args.data_dir / 'mnist'
        mnist = tf.contrib.learn.datasets.DATASETS['mnist'](str(mnist_dir))

        pkl_path = mnist_dir / 'mnist_dict.pkl'
        #print(pkl_path)
        
        if pkl_path.exists():
            mnist_dict = ut.io.load_pickle(pkl_path)
        else:
            # Add defaultdict
            from collections import defaultdict
            #import tqdm
            # Wow
            from tqdm import tqdm
            
            mnist_dict = defaultdict(lambda: defaultdict(list))
            for name in ['train', 'test', 'valid']:
                for num in self.args.mnist_nums:
                    filtered_data = \
                            mnist.train.images[mnist.train.labels == num]
                    filtered_data = \
                            np.reshape(filtered_data, [-1, 28, 28])

                    iterator = tqdm(filtered_data,
                                    desc="[{}] Processing {}".format(name, num))
                    for idx, image in enumerate(iterator):
                        # XXX: don't know which way would be the best
                        resized_image = ut.io.imresize(
                                image, [self.height, self.width],
                                interp='cubic')
                        mnist_dict[name][num].append(
                                np.expand_dims(resized_image, -1))
            ut.io.dump_pickle(pkl_path, mnist_dict)

        mnist_dict = mnist_dict['train' if self.args.train else 'test']

        data = []
        for num in self.args.mnist_nums:
            data.append(mnist_dict[int(num)])

        self.real_data = 255 - np.concatenate([d for d in data])


class SimpleMNIST(MNIST):

    action_sizes = {
            #'pressure': [2],
            'jump': [2],
            #'color': [4],
            #'size': [2],
            'control': None,
            'end': None,
    }

    def __init__(self, args):
        super(SimpleMNIST, self).__init__(args)

## Get arg
> argparser: https://gist.github.com/redja/9276216
>> from config import get_args 

## Feed the arg command and run
> error issues of argparse in jupyter notebook: https://stackoverflow.com/questions/48796169/how-to-fix-ipykernel-launcher-py-error-unrecognized-arguments-in-jupyter
## Tune some of the parameters

In [3]:
if __name__ == '__main__':
    import utils as ut
    # a parser writen by the author
    from config import get_args
    
    # all of the settings depends on the config.py
    args = get_args()

    #####
    # tune some arg parameters in here
    args.env = 'mnist'
    #####
    
    #args = input('Input commands:')
    ut.train.set_global_seed(args.seed)

    env = args.env.lower()
    #env = args.env()
    
    if env == 'mnist':
        env = MNIST(args)
    elif env == 'simple_mnist':
        env = SimpleMNIST(args)
    else:
        raise Exception("Unkown environment: {}".format(args.env))

    for ep_idx in range(10):
        step = 0
        env.reset()

        while True:
            action = env.random_action()
            print("[Step {}] ac: {}".format(
                    step, env.get_action_desc(action)))
            state, reward, terminal, info = env.step(action)
            env.save_image("mnist{}_{}.png".format(ep_idx, step))
            
            if terminal:
                print("Ep #{} finished ==> Reward: {}".format(ep_idx, reward))
                break

            step += 1

08-17 20:32:35 Skip making directories: .data


Extracting .data/mnist/train-images-idx3-ubyte.gz
Extracting .data/mnist/train-labels-idx1-ubyte.gz
Extracting .data/mnist/t10k-images-idx3-ubyte.gz
Extracting .data/mnist/t10k-labels-idx1-ubyte.gz
[Step 0] ac: control: [ 33.03225806  47.48387097] (752)
end: [ 18.58064516  37.16129032] (585)
color: [ 1.          0.87843137  0.4       ] (3)
jump: 1 (1)
pressure: 0.5 (1)
size: 0.2 (0)
[Step 1] ac: control: [ 41.29032258   2.06451613] (52)
end: [ 41.29032258   2.06451613] (52)
color: [ 0.  0.  0.] (0)
jump: 0 (0)
pressure: 0.5 (1)
size: 0.2 (0)
[Step 2] ac: control: [ 53.67741935  26.83870968] (442)
end: [ 41.29032258  10.32258065] (180)
color: [ 0.4         0.85098039  0.90980392] (1)
jump: 0 (0)
pressure: 0.8 (0)
size: 0.7 (1)
[Step 3] ac: control: [ 57.80645161  57.80645161] (924)
end: [ 14.4516129   37.16129032] (583)
color: [ 1.          0.87843137  0.4       ] (3)
jump: 1 (1)
pressure: 0.5 (1)
size: 0.2 (0)
[Step 4] ac: control: [ 57.80645161  49.5483871 ] (796)
end: [  2.06451613  

[Step 4] ac: control: [  4.12903226  12.38709677] (194)
end: [ 61.93548387  64.        ] (1022)
color: [ 0.67843137  0.70980392  0.74117647] (2)
jump: 0 (0)
pressure: 0.8 (0)
size: 0.7 (1)
Ep #8 finished ==> Reward: 0
[Step 0] ac: control: [  6.19354839  10.32258065] (163)
end: [ 12.38709677  53.67741935] (838)
color: [ 0.67843137  0.70980392  0.74117647] (2)
jump: 0 (0)
pressure: 0.8 (0)
size: 0.7 (1)
[Step 1] ac: control: [ 28.90322581  45.41935484] (718)
end: [ 47.48387097  37.16129032] (599)
color: [ 0.4         0.85098039  0.90980392] (1)
jump: 0 (0)
pressure: 0.8 (0)
size: 0.2 (0)
[Step 2] ac: control: [  8.25806452  53.67741935] (836)
end: [ 22.70967742  37.16129032] (587)
color: [ 0.4         0.85098039  0.90980392] (1)
jump: 1 (1)
pressure: 0.8 (0)
size: 0.7 (1)
[Step 3] ac: control: [ 55.74193548  26.83870968] (443)
end: [ 24.77419355  16.51612903] (268)
color: [ 0.67843137  0.70980392  0.74117647] (2)
jump: 1 (1)
pressure: 0.8 (0)
size: 0.7 (1)
[Step 4] ac: control: [ 53.677

## For Tensorboard

In [6]:
'''
logs_path = 'TensorBoard/' # save the tensorboard file in this folder
merged = tf.summary.merge_all() # for tensorboard
writer = tf.summary.FileWriter(logs_path , graph = tf.get_default_graph()) # for tensorboard

sess = tf.Session()
sess.run(tf.initialize_all_variables())

# The following code means we train 600 times and each time we feed in 10 pictures
for i in range(600): # We call it 600 epochs
    batch_xs, batch_ys = mnist.train.next_batch(10)
    sess.run(train_step, feed_dict = {xs: batch_xs, ys: batch_ys, keep_prob:0.5} )
    if i % 100 == 0:
        summary = sess.run(merged, feed_dict = {xs: batch_xs, ys: batch_ys, keep_prob:0.5})
        writer.add_summary(summary, i) # for tensorboard
        
writer.close() # for tensorboard      
sess.close() # for tensorboard
'''

Instructions for updating:
Use `tf.global_variables_initializer` instead.


NameError: name 'mnist' is not defined

## For saving weights(Using Keras)

## Simple drawing agent
> ### Knowing how to control a drawing agent

In [9]:
class Simple(Environment):

    action_sizes = {
            'color': [1],
            'shape': [2],
            #'location': None,
    }

    def __init__(self, args):
        super(Simple, self).__init__(args)

        #assert self.conditional, \
        #        "Don't train a Simple env with --conditional=False"

        # object
        self.object_radius = 3
        self.object_size = self.object_radius * 2
        self.background_color = (255, 255, 255)

        self.colors = [
                (0, 0, 0), # black
                (173, 181, 189), # gray 5
                (255, 224, 102), # yellow 3
                #(102, 217, 232), # cyan 3
        ]
        if 'color' in self.action_sizes:
            self.colors = self.colors[:self.action_sizes['color'][0]]

        self.shapes = ['circle', 'rectangle']
        if 'shape' in self.action_sizes:
            self.shapes = self.shapes[:self.action_sizes['shape'][0]]

        self.locations = utils.uniform_locations(
                self.screen_size, self.location_size,
                self.object_radius)

        self.image = None
        self.drawer = None
        self.random_target = None

    def reset(self):
        self.random_target = self.get_random_target()
        self.image = Image.new(
                'RGB', (self.width, self.height), self.background_color)
        self.drawer = ImageDraw.Draw(self.image)
        self._step = 0

        # TODO(taehoon): z
        self.z = None
        return self.state, self.random_target, self.z

    def draw(self, ac, drawer=None):
        if drawer is None:
            drawer = self.drawer

        r = self.object_radius
        x, y = self.locations[0]
        color, shape = self.colors[0], self.shapes[0]

        for name in self.action_sizes:
            named_ac = ac[self.ac_idx[name]]
            value = getattr(self, name + "s")[named_ac]

            if name == 'location':
                x, y = value
            elif name == 'color':
                color = value
            elif name == 'shape':
                shape = value

        if shape == 'circle':
            drawer.ellipse((x-r, y-r, x+r, y+r), fill=color)
        elif shape == 'rectangle':
            drawer.rectangle((x-r, y-r, x+r, y+r), fill=color)
        else:
            raise Exception("Unkown shape: {}".format(shape))

    def get_random_target(self):
        image = Image.new(
                'RGB', (self.width, self.height), self.background_color)
        drawer = ImageDraw.Draw(image)

        locations = []
        for _ in range(self.episode_length):
            ac = self.random_action(locations=locations)
            self.draw(ac, drawer)
            if 'location' in self.ac_idx:
                locations.append(ac[self.ac_idx['location']])
            else:
                locations.append(self.locations[0])

        return np.array(self.norm(image))

    def random_action(self, locations=[]):
        action = []
        for name  in self.acs:
            size = self.action_sizes[name]
            while True:
                sample = np.random.randint(np.prod(size))
                if name == 'locations':
                    if sample in locations:
                        continue
                else:
                    break
            action.append(sample)
        return action

    def step(self, acs):
        self.draw(acs, self.drawer)
        self._step += 1
        terminal = (self._step == self.episode_length)
        if terminal:
            if self.conditional:
                reward = - utils.l2(self.state, self.random_target) \
                        / np.prod(self.observation_shape) * 100
            else:
                reward = None
        else:
            reward = 0

        # XXX: DEBUG
        if reward == 0: reward = 1

        # state, reward, terminal, info
        return self.state, reward, terminal, {}

    def save_image(self, path):
        self.image.save(path)

    @property
    def state(self):
        return np.array(self.norm(self.image))


if __name__ == '__main__':
    from config import get_args
    args = get_args()
    
    
    '''
    env = args.env.lower()
    if env == 'mnist':
        env = MNIST(args)
    elif env == 'simple_mnist':
        env = SimpleMNIST(args)
    '''
    env = Simple(args)

    for ep_idx in range(10):
        step = 0
        env.reset()

        while True:
            action = env.random_action()
            print("[Step {}] ac: {}".format(step, action))
            state, reward, terminal, info = env.step(action)
            step += 1
            
            if terminal:
                print("Ep #{} finished.".format(ep_idx))
                env.save_image("simple{}.png".format(ep_idx))
                break

[Step 0] ac: [0, 0]
[Step 1] ac: [0, 0]
[Step 2] ac: [0, 0]
[Step 3] ac: [0, 1]
[Step 4] ac: [0, 1]
Ep #0 finished.
[Step 0] ac: [0, 1]
[Step 1] ac: [0, 1]
[Step 2] ac: [0, 0]
[Step 3] ac: [0, 1]
[Step 4] ac: [0, 0]
Ep #1 finished.
[Step 0] ac: [0, 0]
[Step 1] ac: [0, 0]
[Step 2] ac: [0, 1]
[Step 3] ac: [0, 0]
[Step 4] ac: [0, 0]
Ep #2 finished.
[Step 0] ac: [0, 1]
[Step 1] ac: [0, 0]
[Step 2] ac: [0, 1]
[Step 3] ac: [0, 1]
[Step 4] ac: [0, 1]
Ep #3 finished.
[Step 0] ac: [0, 1]
[Step 1] ac: [0, 0]
[Step 2] ac: [0, 1]
[Step 3] ac: [0, 1]
[Step 4] ac: [0, 1]
Ep #4 finished.
[Step 0] ac: [0, 0]
[Step 1] ac: [0, 0]
[Step 2] ac: [0, 0]
[Step 3] ac: [0, 1]
[Step 4] ac: [0, 0]
Ep #5 finished.
[Step 0] ac: [0, 0]
[Step 1] ac: [0, 0]
[Step 2] ac: [0, 1]
[Step 3] ac: [0, 0]
[Step 4] ac: [0, 1]
Ep #6 finished.
[Step 0] ac: [0, 1]
[Step 1] ac: [0, 1]
[Step 2] ac: [0, 0]
[Step 3] ac: [0, 1]
[Step 4] ac: [0, 1]
Ep #7 finished.
[Step 0] ac: [0, 1]
[Step 1] ac: [0, 0]
[Step 2] ac: [0, 1]
[Step 3] ac: